# BLOCKCHAIN IMPLEMENTATION USING SHA-3 ALOGORITHM

>###  Creating Blockchain using Python, mining new blocks, and displaying the whole blockchain: 

- The data will be stored in JSON format which is very easy to implement and easy to read. The data is stored in a block and the block contains multiple data. Each and every minute multiple blocks are added and to differentiate one from the other we will use fingerprinting.

- The fingerprinting is done by using hash and to be particular we will use the SHA3-512 hashing algorithm. Every block will contain its own hash and also the hash of the previous function so that it cannot get tampered with.

- This fingerprinting will be used to chain the blocks together. Every block will be attached to the previous block having its hash and to the next block by giving its hash.

- The mining of the new block is done by giving successfully finding the answer to the proof of work. To make mining hard the proof of work must be hard enough to get exploited.

- After mining the block successfully the block will then be added to the chain.

- After mining several blocks the validity of the chain must be checked in order to prevent any kind of tampering with the blockchain.

- Then the web interface will be made by using Fast api and deployed locally or publicly as per the need of the user.


## Libraries used

In [ ]:
import datetime as dt
import hashlib 
import json

## Creating genesis block
> The genesis block is the first ever block recorded on its respective blockchain network, also occasionally referred to as Block 0 or Block 1. 

> When a block is broadcasted to the blockchain, it references the previous block. However, in the case of the genesis block, there is no previous block to reference.

In [ ]:

    def __init__(self)->None:
        self.chain=list()
        genesis_block=self.create_block(data=input("enter message:",
                                        proof=1,
                                        previous_hash="0",
                                        index=1)
        self.chain.append(genesis_block)
        

## Mining a block –
> Mining a block refers to adding of the block present in the blockchain network. A miner selects a set of transactions from the pool of transactions and then mines the block, or one can say, computes the hash to add the block to the network. If two or more miners mine the same block at the same time, the block with more difficulty is selected. The others are known as stale blocks. Mining usually rewards miners with blockchain currency.

In [ ]:
      
    def mine_block(self, data:str)->dict:
        previous_block=self.get_previous_block()
        previous_proof=previous_block["proof"]
        index=len(self.chain)+1
        proof=self.proof_of_work(previous_proof, index, data)
        previous_hash=self._hash(block=previous_block)
        block=self.create_block(
            data=data, proof=proof, previous_hash=previous_hash, index=index
        )
        self.chain.append(block)
        return block

In [ ]:

    def _hash(self, block:dict)->str:
        encoded_block=json.dumps(block, sort_keys=True).encode()
        return hashlib.sha3_512(encoded_block).hexdigest()
    


## Proof-of-Work Hash Functions
> Blockchain proof-of-work mining algorithms use a special class of hash functions which are computational-intensive and memory-intensive. These hash functions are designed to consume a lot of computational resources and a lot of memory and to be very hard to be implemented in a hardware devices

In [ ]:
  
        
    def to_digest(self, new_proof:int, previous_proof:int, index:str, data:str)->bytes:
        to_digest=str(new_proof**2-previous_proof**2+index)+data
        return to_digest.encode()
    
    
    def proof_of_work(self, previous_proof:str,index:int,data:str)->int:
        new_proof=1
        check_proof=False
        
        while not check_proof:
            to_digest=self.to_digest(new_proof=new_proof,
                                    previous_proof=previous_proof,
                                    index=index,
                                    data=data,
                                    )
            
            hash_value=hashlib.sha3_512(to_digest).hexdigest()
            
            if hash_value[:4]=="0000":
                check_proof=True
            else:
                new_proof+=1
        return new_proof
    

In [ ]:

    
    def get_previous_block(self)->dict:
        return self.chain[-1]
    
    def create_block(self, data: str, proof:int, previous_hash:str, index:int)->dict:
        block={
            "index":index,
            "timestamp":str(dt.datetime.now()),
            "data":data,
            "proof":proof,
            "previous_hash":previous_hash
        }
        
    
 

## Validating a New Block
> When a node receives a new block, it will validate the block by checking it against a long list of criteria that must all be met; otherwise, the block is rejected.

In [ ]:
   
    def is_chain_valid(self)->bool:
        current_block=self.chain[0]
        block_index=1
        
        while block_index<len(self.chain):
            next_block=self.chain[block_index]
            
            if next_block["previous_hash"]!= self._hash(current_block):
                return False
            
            current_proof=current_block["proof"]
            next_index, next_data, next_proof=(next_block["index"],next_block["data"],next_block["proof"],)
            
            
            hash_value=hashlib.sha3_512(
                self.to_digest(new_proof=next_proof,previous_proof=current_proof, index=next_index, data=next_data,
                              )
            ).hexdigest()
            if hash_value[:4]!="0000":# first 4 bits of hash must be 0000
                return False
            
            current_block=next_block
            block_index+=1
            
        return True
